In [1]:
import os
os.chdir(r"C:\Users\Laura\Desktop\saeb_microdados_dashboard")

In [2]:
import pandas as pd
from src.ingestion.read_alunos97 import read_alunos97
from src.cleaning.alunos_cleaning import clean_alunos_data_minimal
from src.dictionaries.alunos_maps import (
    map_categorias_basicas_alunos,
    map_geografia_alunos,
    map_questionario_socioeconomico,
    map_opiniao_matematica,
    map_data_nascimento_visual
    
)

In [3]:
df_alunos = read_alunos97(1997)
pd.set_option("display.max_rows", None)

df_alunos = map_geografia_alunos(df_alunos)
df_alunos = map_categorias_basicas_alunos(df_alunos)
df_alunos = map_data_nascimento_visual(df_alunos)
df_alunos = map_questionario_socioeconomico(df_alunos)

In [ ]:

import pandas as pd
from src.ingestion.read_alunos95 import read_alunos95
from src.cleaning.alunos_cleaning import clean_alunos_data_minimal
from src.dictionaries.alunos_maps import (
    map_categorias_basicas_alunos,
    map_geografia_alunos,
    map_questionario_socioeconomico,
    map_opiniao_matematica,
    map_data_nascimento_visual
    
)


In [ ]:
df_alunos = read_alunos95(1995)
pd.set_option("display.max_rows", None)

df_alunos = map_categorias_basicas_alunos(df_alunos)
df_alunos = map_geografia_alunos(df_alunos)
df_alunos = map_questionario_socioeconomico(df_alunos)
df_alunos = map_opiniao_matematica(df_alunos)
df_alunos = map_data_nascimento_visual(df_alunos)
df_alunos = clean_alunos_data_minimal(df_alunos)



In [4]:
df_alunos.head()

,mascara,ano_mascara,ano,serie,disciplina,turma,turno,id_aluno,idade,estrato,upa,dep_adm,local,rede,uf,regiao,peso_ac,caderno,resp_bl1,resp_bl2,resp_bl3,bloco1,bloco2,bloco3,gab_bl1,gab_bl2,gab_bl3,proficiencia,data_nascimento,idade_entrada_fundamental,sexo,raca,estado_civil,mora_com,instrucao_pai,instrucao_mae,pai_trabalha,mae_trabalha,situacao_economica,horas_trabalho,tipo_escola,interrompeu_estudos,motivo_interrupcao,repetiu_ano,turnos_estudados,supletivo,faltas_ano,motivo_faltas,tempo_casa_escola,conversa_escola,reuniao_escola,compra_jornal,le_jornal,horas_tv,programacao_tv,tempo_estudo_fora,sabe_computador,computador_casa,computador_escola,motivo_na_escola,como_estuda,opiniao_teste,tempo_licao_mat,jornais_revistas_licao,leitura_indicada,ajuda_duvidas_mat,gosta_mat,professor_mat_inicio,prejuizo_falta,mat_dia_a_dia,esforco_professor,mat_ajuda_outras,outros_espacos,entende_professor,auto_desempenho_mat,faltas
0,20021521,1997,1997,3º Ano,M,1,1,4,16.0,3509,351506148708,Estadual,Urbana,Pública,São Paulo,Sudeste,726.7513,11,CEDEECBDAA,CDBCCAEDCE,DAABABEAAE,11,12,2,CEDEEDAADA,CDCCDAEDCE,BBDBAEDADE,383.53911,Não Informado,3.0,Feminino,Branco(a),1.0,1.0,Primário (1ª-4ª),Primário (1ª-4ª),1.0,1.0,1.0,5.0,1.0,6.0,6.0,1.0,1.0,1.0,4.0,1.0,3.0,1.0,2.0,3.0,3.0,2.0,4.0,2.0,1.0,2.0,3.0,6.0,1.0,4.0,3.0,2.0,3.0,3.0,2.0,2.0,1.0,2.0,1.0,2.0,2.0,1.0,2.0,NaN
1,20021521,1997,1997,3º Ano,M,1,1,9,17.0,3509,351506148708,Estadual,Urbana,Pública,São Paulo,Sudeste,726.7513,10,DDBBCDEBAD,CEDEEDDADA,BABCCDBDBB,10,11,1,DDBDBDEBAD,CEDEEDAADA,EDDBECBBDB,361.25445,Não Informado,3.0,Masculino,Pardo(a),1.0,1.0,Superior,Colegial (2º Grau),1.0,1.0,1.0,5.0,1.0,6.0,6.0,1.0,1.0,2.0,3.0,1.0,2.0,3.0,1.0,1.0,1.0,3.0,5.0,1.0,2.0,2.0,2.0,6.0,4.0,4.0,2.0,3.0,3.0,1.0,1.0,1.0,2.0,1.0,1.0,2.0,3.0,1.0,1.0,NaN
2,20021521,1997,1997,3º Ano,M,1,1,14,19.0,3509,351506148708,Estadual,Urbana,Pública,São Paulo,Sudeste,726.7513,9,EEADBDADBC,CCBBBCEBAC,CADBCBDAAE,9,10,13,DDBABDEDDC,DDBDBDEBAD,BCDABBCACA,286.75193,Não Informado,4.0,Feminino,Pardo(a),1.0,1.0,Primário (1ª-4ª),Primário (1ª-4ª),1.0,1.0,2.0,2.0,3.0,5.0,6.0,3.0,3.0,2.0,3.0,1.0,1.0,1.0,2.0,3.0,2.0,1.0,4.0,1.0,1.0,2.0,3.0,6.0,2.0,3.0,4.0,2.0,3.0,1.0,3.0,1.0,2.0,1.0,1.0,2.0,3.0,3.0,3.0,NaN
3,20021521,1997,1997,3º Ano,M,1,1,19,17.0,3509,351506148708,Estadual,Urbana,Pública,São Paulo,Sudeste,726.7513,8,BCEDDAACDB,DDAACDDDDB,CACDDACCCC,8,9,12,BDDBBBBCBC,DDBABDEDDC,CDCCDAEDCE,292.31713,Não Informado,4.0,Masculino,Branco(a),1.0,1.0,NaN,NaN,2.0,4.0,2.0,1.0,1.0,6.0,6.0,1.0,1.0,1.0,3.0,3.0,1.0,5.0,1.0,3.0,3.0,2.0,5.0,3.0,1.0,1.0,3.0,3.0,7.0,4.0,3.0,3.0,3.0,1.0,2.0,1.0,3.0,1.0,3.0,2.0,3.0,3.0,2.0,NaN
4,20021521,1997,1997,3º Ano,M,1,1,24,17.0,3509,351506148708,Estadual,Urbana,Pública,São Paulo,Sudeste,726.7513,7,ABBECBDCAC,DDDBBDACEB,BCBCCBEDAA,7,8,11,CBACECCBCE,BDDBBBBCBC,CEDEEDAADA,221.77330,Não Informado,3.0,Feminino,Amarelo(a),1.0,3.0,Ginásio (5ª-8ª),Ginásio (5ª-8ª),4.0,1.0,2.0,1.0,3.0,6.0,6.0,1.0,1.0,2.0,3.0,2.0,1.0,1.0,2.0,1.0,3.0,2.0,4.0,2.0,2.0,1.0,3.0,6.0,6.0,1.0,3.0,3.0,2.0,3.0,2.0,2.0,1.0,2.0,2.0,3.0,3.0,2.0,3.0,NaN


In [ ]:
df_alunos[df_alunos['uf']=="Acre"]['dep_adm'].value_counts()

In [ ]:
df_alunos.info()

In [ ]:
import plotly.express as px
import streamlit as st

# 1. Agrupar os dados pela média de proficiência
df_comparativo = df_alunos.groupby("dep_adm")["proficiencia"].mean().reset_index()

# 2. Ordenar para o gráfico ficar mais intuitivo (opcional)
df_comparativo = df_comparativo.sort_values("proficiencia", ascending=False)

In [ ]:
# Criando a figura
fig = px.bar(
    df_comparativo, 
    x="dep_adm", 
    y="proficiencia",
    title="Média de Proficiência por Dependência Administrativa",
    labels={"dep_adm": "Dependência Adm.", "proficiencia": "Proficiência Média"},
    color="proficiencia", # Muda a cor baseado no valor
    color_continuous_scale="Viridis",
    text_auto='.1f' # Exibe o valor em cima da barra com 1 casa decimal
)

# Ajustando o layout
fig.update_layout(xaxis_title="Rede de Ensino", yaxis_title="Proficiência (Média)")

# Exibindo no Streamlit
st.plotly_chart(fig, use_container_width=True)

In [ ]:
ufs = st.multiselect("Selecione os Estados", options=df_alunos['uf'].unique())
if ufs:
    df_filtrado = df_alunos[df_alunos['uf'].isin(ufs)]
else:
    df_filtrado = df_alunos